# Cuaderno 01: Generación de Datasets y Embeddings

### Objetivo 🎯
Este cuaderno es el punto de partida fundamental de nuestro proyecto. Su objetivo es construir los **datasets paralelos** (auditivo y visual) y las **representaciones numéricas (embeddings)** que servirán de base para el entrenamiento de los modelos.

### Flujo de Trabajo  Workflow ⚙️

1.  **Configuración e Instalación**: Definiremos los parámetros globales, las listas de unidades lingüísticas (fonemas y palabras) y las rutas de salida. También instalaremos las librerías necesarias.
2.  **Generación de Audio para Fonemas**: Crearemos archivos `.wav` para cada fonema/grafema de nuestras listas utilizando la librería `gTTS`.
3.  **Generación de Audio para Palabras**: Cargaremos listas de palabras desde archivos de diccionario y generaremos sus correspondientes archivos `.wav`.
4.  **Generación del Dataset Visual (EMNIST)**: Descargaremos y extraeremos imágenes de letras (`.png`) del dataset EMNIST que correspondan a nuestros grafemas.
5.  **Extracción de Embeddings Auditivos (Wav2Vec2)**: Procesaremos **todos** los archivos `.wav` generados (tanto fonemas como palabras) con el modelo `Wav2Vec2` para convertirlos en secuencias de vectores (embeddings) y guardarlos como archivos `.npy`.
6.  **Resumen**: Al final de cada paso, se mostrará una tabla de resumen con el estado de la generación de archivos.

## Paso 1: Configuración e Instalación

Primero, instalamos y luego importamos todas las librerías que usaremos en el cuaderno. Separamos la configuración de los parámetros y rutas para mantener el código limpio y modular.

In [ ]:
%pip install gTTS pydub torch torchvision matplotlib transformers librosa protobuf pandas --quiet

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Librerías estándar de Python
import os
from pathlib import Path

# Librerías para generación de audio
from gtts import gTTS
from pydub import AudioSegment
import librosa

# Librerías para manejo de imágenes y datasets
import torch
import torchvision
from torchvision.datasets import EMNIST
from PIL import Image

# Librerías para modelos de audio y arrays numéricos
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Model
import numpy as np

# Librería para manejo y visualización de datos tabulares
import pandas as pd

## Paso 1.2: Configuración, Rutas y Parámetros

Con las dependencias instaladas, el siguiente paso es configurar nuestro entorno de trabajo. Esta celda se encarga de:

* **Importar las librerías** necesarias para el resto del cuaderno.
* **Definir las rutas de archivos**: Se establece una estructura de carpetas organizada para los datos brutos (`01_raw`) y los datos procesados (`02_processed`), separando los datasets auditivos y visuales.
* **Establecer las listas de fonemas/grafemas**: Se definen las unidades fonéticas y gráficas para español e inglés que se generarán en las Partes A y B.
* **Fijar parámetros de procesamiento**: Como el número máximo de imágenes a extraer por cada grafema (`MAX_IMAGES_PER_LETTER`).

In [7]:
# --- Parámetros Globales del Experimento ---
MAX_IMAGES_PER_LETTER = 50
LANGUAGES = ['es', 'en']
WAV2VEC2_MODEL_NAME = "facebook/wav2vec2-large-xlsr-53"
TARGET_SAMPLE_RATE = 16000

# --- Rutas del Proyecto ---
project_root = Path.cwd().parent

# Directorios de datos crudos (entrada)
raw_emnist_dir = project_root / "data/01_raw/emnist"
dictionaries_dir = project_root / "data/01_raw/dictionaries" # Ruta para los diccionarios

# Directorios de datos procesados (salida)
output_audio_dir = project_root / "data/02_processed/phoneme_audio"
output_word_audio_dir = project_root / "data/02_processed/word_audio" # Nueva ruta para audio de palabras
output_visual_dir = project_root / "data/02_processed/grapheme_images"
output_embedding_dir = project_root / "data/02_processed/wav2vec2_embeddings"
output_word_embedding_dir = project_root / "data/02_processed/word_embeddings" # Nueva ruta para embeddings de palabras

# Crear todos los directorios de salida
for p in [output_audio_dir, output_word_audio_dir, output_visual_dir, output_embedding_dir, output_word_embedding_dir, raw_emnist_dir, dictionaries_dir]:
    p.mkdir(parents=True, exist_ok=True)
    
# --- Listas de Fonemas/Grafemas ---
PHONEMES_GRAPHEMES = {
    'es': ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'v', 'w', 'x', 'y', 'z'],
    'en': ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'v', 'w', 'x', 'y', 'z']
}

# --- Carga de Listas de Palabras desde Diccionarios ---
# Se asume que existen archivos de texto en la carpeta de diccionarios
# con una palabra por línea.
WORDS = {}
for lang in LANGUAGES:
    dict_path = dictionaries_dir / f"{lang}_words.txt"
    try:
        with open(dict_path, 'r', encoding='utf-8') as f:
            WORDS[lang] = [line.strip() for line in f if line.strip()]
        print(f"Diccionario para '{lang.upper()}' cargado con {len(WORDS[lang])} palabras.")
    except FileNotFoundError:
        print(f"ADVERTENCIA: No se encontró el archivo de diccionario en '{dict_path}'. La generación de audio para palabras de '{lang.upper()}' será omitida.")
        WORDS[lang] = []

Diccionario para 'ES' cargado con 100 palabras.
Diccionario para 'EN' cargado con 100 palabras.


## Paso 2: Generación de Audio para Fonemas (TTS)

Primero, generamos los archivos de audio para las unidades fonéticas/grafémicas básicas definidas en nuestras listas.

In [8]:
def generate_audio_files(item_list, lang_code, output_path, task_name=""):
    """Sintetiza y guarda archivos WAV para una lista de ítems (fonemas o palabras).

    Args:
        item_list (list): Lista de strings a sintetizar.
        lang_code (str): Código de idioma para gTTS (ej. 'es', 'en').
        output_path (Path): Directorio donde guardar los archivos .wav.
        task_name (str): Nombre descriptivo para la impresión en pantalla.
    """
    print(f"\n--- Iniciando generación de audio para: {task_name} ({lang_code.upper()}) ---")
    output_path.mkdir(parents=True, exist_ok=True)
    
    for i, item in enumerate(item_list):
        wav_filepath = output_path / f"{item}.wav"
        if wav_filepath.exists():
            print(f"({i+1}/{len(item_list)}) Audio para '{item}' ya existe. Omitiendo.")
            continue
        
        try:
            tts = gTTS(text=item, lang=lang_code, slow=True)
            mp3_temp_path = output_path / "_temp.mp3"
            tts.save(mp3_temp_path)
            
            audio = AudioSegment.from_mp3(mp3_temp_path)
            audio = audio.set_channels(1).set_frame_rate(16000)
            audio.export(wav_filepath, format="wav")
            
            os.remove(mp3_temp_path)
            print(f"({i+1}/{len(item_list)}) Audio para '{item}' guardado.")
        except Exception as e:
            print(f"Error procesando '{item}': {e}")
            if 'mp3_temp_path' in locals() and mp3_temp_path.exists():
                os.remove(mp3_temp_path)

In [9]:
# Ejecutar la generación para cada idioma
for lang in LANGUAGES:
    generate_audio_files(
        item_list=PHONEMES_GRAPHEMES[lang],
        lang_code=lang,
        output_path=output_audio_dir / lang,
        task_name="Fonemas"
    )

print("\n--- Proceso de generación de audio para fonemas completado. ---")


--- Iniciando generación de audio para: Fonemas (ES) ---
(1/25) Audio para 'a' guardado.
(2/25) Audio para 'b' guardado.
(3/25) Audio para 'c' guardado.
(4/25) Audio para 'd' guardado.
(5/25) Audio para 'e' guardado.
(6/25) Audio para 'f' guardado.
(7/25) Audio para 'g' guardado.
(8/25) Audio para 'h' guardado.
(9/25) Audio para 'i' guardado.
(10/25) Audio para 'j' guardado.
(11/25) Audio para 'k' guardado.
(12/25) Audio para 'l' guardado.
(13/25) Audio para 'm' guardado.
(14/25) Audio para 'n' guardado.
(15/25) Audio para 'o' guardado.
(16/25) Audio para 'p' guardado.
(17/25) Audio para 'q' guardado.
(18/25) Audio para 'r' guardado.
(19/25) Audio para 's' guardado.
(20/25) Audio para 't' guardado.
(21/25) Audio para 'v' guardado.
(22/25) Audio para 'w' guardado.
(23/25) Audio para 'x' guardado.
(24/25) Audio para 'y' guardado.
(25/25) Audio para 'z' guardado.

--- Iniciando generación de audio para: Fonemas (EN) ---
(1/25) Audio para 'a' guardado.
(2/25) Audio para 'b' guardado.
(3/2

## Paso 3: Generación de Audio para Palabras (TTS)

Ahora, aplicamos la misma función de generación de audio a las listas de palabras que cargamos desde los archivos de diccionario.

In [11]:
# Ejecutar la generación para las palabras de cada idioma
for lang in LANGUAGES:
    if WORDS[lang]: # Solo ejecutar si la lista de palabras no está vacía
        generate_audio_files(
            item_list=WORDS[lang],
            lang_code=lang,
            output_path=output_word_audio_dir / lang,
            task_name="Palabras"
        )

print("\n--- Proceso de generación de audio para palabras completado. ---")


--- Iniciando generación de audio para: Palabras (ES) ---
(1/100) Audio para 'casa' guardado.
(2/100) Audio para 'perro' guardado.
(3/100) Audio para 'gato' guardado.
(4/100) Audio para 'sol' guardado.
(5/100) Audio para 'luna' guardado.
(6/100) Audio para 'agua' guardado.
(7/100) Audio para 'mesa' guardado.
(8/100) Audio para 'silla' guardado.
(9/100) Audio para 'libro' guardado.
(10/100) Audio para 'mano' guardado.
(11/100) Audio para 'niño' guardado.
(12/100) Audio para 'niña' guardado.
(13/100) Audio para 'pan' guardado.
(14/100) Audio para 'leche' guardado.
(15/100) Audio para 'escuela' guardado.
(16/100) Audio para 'coche' guardado.
(17/100) Audio para 'flor' guardado.
(18/100) Audio para 'árbol' guardado.
(19/100) Audio para 'amigo' guardado.
(20/100) Audio para 'amor' guardado.
(21/100) Audio para 'día' guardado.
(22/100) Audio para 'noche' guardado.
(23/100) Audio para 'luz' guardado.
(24/100) Audio para 'color' guardado.
(25/100) Audio para 'feliz' guardado.
(26/100) Audio p

## Paso 4: Generación del Dataset Visual (EMNIST)

Para la contraparte visual, usamos el dataset EMNIST para obtener imágenes de letras individuales. Esto nos servirá para entrenar la vía visual en el Cuaderno 03.

In [14]:
def generate_visual_dataset(phoneme_list, emnist_dataset, output_path, max_images):
    """Filtra EMNIST y guarda un número máximo de imágenes por grafema."""
    class_to_char = {i: chr(ord('a') + i) for i in range(26)}
    char_to_class = {c: i for i, c in class_to_char.items()}

    single_char_graphemes = sorted([p for p in phoneme_list if len(p) == 1 and p.lower() in char_to_class])
    print(f"\n--- Extrayendo un máximo de {max_images} imágenes para {len(single_char_graphemes)} grafemas de un solo carácter ---")

    images_by_label = {i: [] for i in range(len(emnist_dataset.classes))}
    for image, label in emnist_dataset:
        # CORRECCIÓN: Se quita .item() porque 'label' ya es un entero.
        images_by_label[label - 1].append(image)

    for i, grapheme in enumerate(single_char_graphemes):
        class_idx = char_to_class.get(grapheme.lower())
        if class_idx is not None:
            grapheme_dir = output_path / grapheme
            grapheme_dir.mkdir(parents=True, exist_ok=True)
            
            num_existing = len(list(grapheme_dir.glob("*.png")))
            if num_existing >= max_images:
                print(f"({i+1}/{len(single_char_graphemes)}) Límite para '{grapheme}' ya alcanzado. Omitiendo.")
                continue

            images_to_save = images_by_label[class_idx][:max_images - num_existing]
            
            for j, img_obj in enumerate(images_to_save):
                img = img_obj.transpose(Image.Transpose.ROTATE_270).transpose(Image.Transpose.FLIP_LEFT_RIGHT)
                img.save(grapheme_dir / f"{grapheme}_{num_existing + j}.png")
            
            print(f"({i+1}/{len(single_char_graphemes)}) Se guardaron {len(images_to_save)} imágenes para '{grapheme}'.")
        else:
            print(f"Grafema '{grapheme}' no encontrado en EMNIST 'letters'.")

In [15]:
# Descargar EMNIST una sola vez
print("\n--- Descargando y preparando el dataset EMNIST ---")
emnist_dataset = EMNIST(root=raw_emnist_dir, split='letters', download=True)

# Ejecutar la generación para cada idioma
for lang in LANGUAGES:
    generate_visual_dataset(
        phoneme_list=PHONEMES_GRAPHEMES[lang],
        emnist_dataset=emnist_dataset,
        output_path=output_visual_dir / lang,
        max_images=MAX_IMAGES_PER_LETTER
    )

print("\n--- Proceso de generación de imágenes completado. ---")


--- Descargando y preparando el dataset EMNIST ---

--- Extrayendo un máximo de 50 imágenes para 25 grafemas de un solo carácter ---
(1/25) Se guardaron 50 imágenes para 'a'.
(2/25) Se guardaron 50 imágenes para 'b'.
(3/25) Se guardaron 50 imágenes para 'c'.
(4/25) Se guardaron 50 imágenes para 'd'.
(5/25) Se guardaron 50 imágenes para 'e'.
(6/25) Se guardaron 50 imágenes para 'f'.
(7/25) Se guardaron 50 imágenes para 'g'.
(8/25) Se guardaron 50 imágenes para 'h'.
(9/25) Se guardaron 50 imágenes para 'i'.
(10/25) Se guardaron 50 imágenes para 'j'.
(11/25) Se guardaron 50 imágenes para 'k'.
(12/25) Se guardaron 50 imágenes para 'l'.
(13/25) Se guardaron 50 imágenes para 'm'.
(14/25) Se guardaron 50 imágenes para 'n'.
(15/25) Se guardaron 50 imágenes para 'o'.
(16/25) Se guardaron 50 imágenes para 'p'.
(17/25) Se guardaron 50 imágenes para 'q'.
(18/25) Se guardaron 50 imágenes para 'r'.
(19/25) Se guardaron 50 imágenes para 's'.
(20/25) Se guardaron 50 imágenes para 't'.
(21/25) Se guar

## Paso 5: Extracción de Embeddings Auditivos (Wav2Vec2)

Ahora que tenemos todos los archivos de audio (fonemas y palabras), los convertimos en **embeddings** usando `Wav2Vec2`. Este paso es crucial, ya que transforma el sonido en una representación vectorial que los modelos de deep learning pueden procesar.

In [16]:
# --- Carga del Modelo y Extractor de Características ---
print("Cargando el modelo Wav2Vec2... (esto puede tardar la primera vez)")
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(WAV2VEC2_MODEL_NAME)
model_wav2vec = Wav2Vec2Model.from_pretrained(WAV2VEC2_MODEL_NAME)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_wav2vec.to(device)
print(f"Modelo cargado y movido al dispositivo: {device}")

Cargando el modelo Wav2Vec2... (esto puede tardar la primera vez)
Modelo cargado y movido al dispositivo: cuda


In [17]:
def extract_embeddings(audio_dir, embedding_dir, task_name=""):
    """
    Procesa una carpeta de archivos .wav y guarda sus embeddings como .npy.
    """
    print(f"\n--- Extrayendo embeddings para: {task_name} ---")
    embedding_dir.mkdir(parents=True, exist_ok=True)
    audio_files = sorted(list(audio_dir.glob("*.wav")))
    
    for i, path in enumerate(audio_files):
        label = path.stem
        embedding_path = embedding_dir / f"{label}.npy"
        if embedding_path.exists():
            print(f"({i+1}/{len(audio_files)}) Embedding para '{label}' ya existe. Omitiendo.")
            continue
        try:
            waveform, _ = librosa.load(path, sr=TARGET_SAMPLE_RATE)
            inputs = feature_extractor(waveform, sampling_rate=TARGET_SAMPLE_RATE, return_tensors="pt", padding=True).input_values.to(device)
            
            with torch.no_grad():
                outputs = model_wav2vec(inputs)
            
            sequence_embedding = outputs.last_hidden_state.squeeze(0).cpu().numpy()
            np.save(embedding_path, sequence_embedding)
            print(f"({i+1}/{len(audio_files)}) Embedding para '{label}' guardado.")
        except Exception as e:
            print(f"Error procesando '{label}': {e}")

In [18]:
# --- Ejecutar Extracción para Todos los Datasets Auditivos ---
# Fonemas
for lang in LANGUAGES:
    extract_embeddings(
        audio_dir=output_audio_dir / lang,
        embedding_dir=output_embedding_dir / lang,
        task_name=f"Fonemas ({lang.upper()})"
    )

# Palabras
for lang in LANGUAGES:
    if WORDS[lang]:
        extract_embeddings(
            audio_dir=output_word_audio_dir / lang,
            embedding_dir=output_word_embedding_dir / lang,
            task_name=f"Palabras ({lang.upper()})"
        )

print("\n--- Extracción de embeddings completada. ---")


--- Extrayendo embeddings para: Fonemas (ES) ---
(1/25) Embedding para 'a' guardado.
(2/25) Embedding para 'b' guardado.
(3/25) Embedding para 'c' guardado.
(4/25) Embedding para 'd' guardado.
(5/25) Embedding para 'e' guardado.
(6/25) Embedding para 'f' guardado.
(7/25) Embedding para 'g' guardado.
(8/25) Embedding para 'h' guardado.
(9/25) Embedding para 'i' guardado.
(10/25) Embedding para 'j' guardado.
(11/25) Embedding para 'k' guardado.
(12/25) Embedding para 'l' guardado.
(13/25) Embedding para 'm' guardado.
(14/25) Embedding para 'n' guardado.
(15/25) Embedding para 'o' guardado.
(16/25) Embedding para 'p' guardado.
(17/25) Embedding para 'q' guardado.
(18/25) Embedding para 'r' guardado.
(19/25) Embedding para 's' guardado.
(20/25) Embedding para 't' guardado.
(21/25) Embedding para 'v' guardado.
(22/25) Embedding para 'w' guardado.
(23/25) Embedding para 'x' guardado.
(24/25) Embedding para 'y' guardado.
(25/25) Embedding para 'z' guardado.

--- Extrayendo embeddings para: F

## Paso 6: Conclusión y Próximos Pasos

Hemos generado con éxito todos los datos de entrada necesarios para nuestros modelos:
-   ✅ **Archivos de Audio**: `.wav` para fonemas y palabras.
-   ✅ **Embeddings Auditivos**: Archivos `.npy` para fonemas y palabras.
-   ✅ **Imágenes de Grafemas**: Carpetas de imágenes `.png` para las letras.

Con estos datos listos, podemos proceder a los cuadernos de entrenamiento.